In [ ]:
#Import standard packages for model training

import tensorflow as T
from tensorflow import *
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.losses import *
import numpy as np
import pandas as pd
import sklearn.metrics as SK
from sklearn.metrics import *
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import io
from utils.GPU import *
from utils import utils
commons = utils.Commons()
ts_helper = utils.TS_Helper()
ts_helper.model_type = ts_helper.Regression
shap_helper = utils.Shap_Helper()

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [ ]:
#Inport train, validation and test sets

training = './data/regression/Tb.rhodesiense/random_split/train_fold_4.csv'
validation = './data/regression/Tb.rhodesiense/random_split/valid_fold_4.csv'
test = './data/regression/Tb.rhodesiense/random_split/test_fold_4.csv'
SMILES = 'SMILES'
TASK_START = 2
NUM_TASKS = 1
train_dataset,y_train,train_smiles = commons.load_dataset(training,SMILES,TASK_START,NUM_TASKS)
valid_dataset,y_val,val_smiles = commons.load_dataset(validation,SMILES,TASK_START,NUM_TASKS)
test_dataset,y_test,test_smiles = commons.load_dataset(test,SMILES,TASK_START,NUM_TASKS)


train_dataset.head()

In [ ]:
# calculate ECFP (defaut) fingerprints using RDKit
FP_SIZE = 2048
RADIUS = 2
FEAT = False
X_train = commons.assing_fp(train_smiles,FP_SIZE,RADIUS,FEAT)
X_val = commons.assing_fp(val_smiles,FP_SIZE,RADIUS,FEAT)
X_test = commons.assing_fp(test_smiles,FP_SIZE,RADIUS,FEAT)

type(X_train), X_val.shape, X_test.shape

In [ ]:
#Parameters for train network

from utils.utils import *

bit_vector = X_train.shape[1]

def create_model():
    return T.keras.models.Sequential([
            T.keras.layers.Dense(15,input_dim=bit_vector, activation='relu'),
            T.keras.layers.Dropout(0.3),
            T.keras.layers.Dense(10, activation='relu'),
            T.keras.layers.Dropout(0.1),
            T.keras.layers.Dense(10, activation='relu'),
            T.keras.layers.Dropout(0.1),
            T.keras.layers.Dense(NUM_TASKS, activation='linear'),
            ])

       
optimizer = RMSprop(lr=0.001)
lr_metric = ts_helper.get_lr_metric(optimizer)
model = create_model()
model.compile(loss=ts_helper.regression_loss, metrics = [lr_metric])    
model.summary()
T.keras.utils.plot_model(model)

In [ ]:
# Early stopping parameters

callbacks_list = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00000001, verbose=1, mode='auto',cooldown=0),
    ModelCheckpoint(filepath="./data/TM-FNN_regression_model.hdf5", monitor='val_loss', save_best_only=True, verbose=1, mode='auto'),
    EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, mode='min', verbose=1)
    ]


# parameters for train network

epochs=1000
batch_size=40

history = model.fit(X_train, y_train, 
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val),
                    callbacks=(callbacks_list))

In [ ]:
#Plot model history
ts_helper.plot_history(history)

In [ ]:
#Statistical characteristico of model without 3-sigma rule
THRESHOLD = 0.5
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)
ts_helper.get_DLmodelStats(model, X_train, y_train, X_test, y_test,X_val, y_val,NUM_TASKS,THRESHOLD)